In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.mode.chained_assignment = None

In [44]:
avalanches_df = pd.read_csv('CAIC_avalanches.csv')
avalanches_df = avalanches_df[["Date", "BC Zone", "Asp", "Type", "Trigger", "Rsize", "Dsize", "Area Description", "Comments"]]
avalanches_df.fillna(value = {"Date": pd.NA, "BC Zone": pd.NA, "Asp": pd.NA, "Type": pd.NA, "Trigger": pd.NA, "Area Description": "", "Comments": ""}, inplace = True)
avalanches_df["Description"], avalanches_df["Zone"], avalanches_df["Aspect"] = avalanches_df[["Area Description", "Comments"]].agg('\n'.join, axis=1).str.strip(), avalanches_df["BC Zone"], avalanches_df["Asp"]
avalanches_df.drop(labels = ["Area Description", "Comments", "BC Zone", "Asp"], axis = 1, inplace = True)
avalanches_df["Date"] = pd.to_datetime(avalanches_df["Date"])
avalanches_df["Rsize"], avalanches_df["Dsize"] = avalanches_df["Rsize"].apply(lambda x: x[1:] if type(x) == str else x), avalanches_df["Dsize"].apply(lambda x: x[1:] if type(x) == str else x)
avalanches_df["Rsize"], avalanches_df["Dsize"] = avalanches_df.Rsize.apply(lambda x: str(x) if pd.notna(x) else x).replace('', np.nan), avalanches_df.Dsize.apply(lambda x: str(x) if pd.notna(x) else x).replace('', np.nan)
avalanches_df["Rsize"].fillna(avalanches_df["Rsize"].astype(float).mean(skipna = True), inplace = True)
avalanches_df["Dsize"].fillna(avalanches_df["Dsize"].astype(float).mean(skipna = True), inplace = True)
avalanches_df["Rsize"], avalanches_df["Dsize"] = avalanches_df["Rsize"].astype(float), avalanches_df["Dsize"].astype(float)
avalanches_df.dtypes

,Date,Type,Trigger,Rsize,Dsize,Description,Zone,Aspect
0,2021-11-11,SS,AX,2.000000,1.500000,,Front Range,N
1,2021-11-11,SS,AX,2.000000,1.500000,,Front Range,N
2,2021-11-11,SS,AX,1.000000,1.000000,,Front Range,N
3,2021-11-11,SS,AX,2.000000,1.500000,,Front Range,N
4,2021-11-11,SS,AX,2.000000,2.000000,,Front Range,N
...,...,...,...,...,...,...,...,...
18298,2010-05-08,<NA>,<NA>,1.896957,1.765915,,Front Range,E
18299,2010-05-07,SS,N,3.000000,2.000000,recent with small trees in the debris,Vail & Summit County,NE
18300,2010-05-07,WL,N,1.000000,1.765915,,Vail & Summit County,N
18301,2010-05-07,<NA>,<NA>,1.896957,1.765915,,Front Range,SE
